In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time


# headless option
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

driver = webdriver.Chrome('../web-drivers/chromedriver', options=options)

url = 'https://brunch.co.kr/keyword/IT_트렌드?q=g'
driver.get(url)
elem = driver.find_element_by_tag_name('body')

for i in range(5):
    elem.send_keys(Keys.END)
    time.sleep(1)
    
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
a_tags = soup.body.find_all('a', {'class': 'link_post'})

driver.quit()

In [6]:
from datetime import datetime
from pytz import timezone
from typing import List 
import bs4

Tag = bs4.element.Tag

def is_today(date_str: str) -> List[Tag] :
    # *시간전 or *분전
    if len(date_str) < 6:
        return True

    KST = timezone('Asia/Seoul')
    today = datetime.now(KST).strftime('%d')
    pub = datetime.strptime(date_str, '%b %d. %Y').strftime('%d')
    return pub == today

In [7]:
KST = timezone('Asia/Seoul')
issue_title = "[IT 트렌드] Today's brunch - " + datetime.now(KST).strftime('%Y.%m.%d')
issue_body = ''
for e in a_tags:
    publish_time = e.find('span', {'class': 'publish_time'}).text
    if not is_today(publish_time):
        break

    href = 'https://brunch.co.kr' + e['href']
    author = 'by ' + e.find_all('span', {'class': 'name_txt'})[-1].text
    title = e.find('strong').text
    content = e.find('span', {'class': 'article_content'}).text + ' ...'
    issue_body += '<h1>' + title + '</h1><a href="' + href + '">' + href + '</a><br><br>' + content + '<h3><p align="right">' + author + '</h3></p><br><br><br>'

In [5]:
import os
import sys
from github import Github

if issue_body == '':
    sys.exit()

try:
    GITHUB_TOKEN = os.environ['GITHUB_TOKEN']
    REPO_NAME = 'todays-brunch'

    repo = Github(GITHUB_TOKEN).get_user().get_repo(REPO_NAME)
    res = repo.create_issue(title=issue_title, body=issue_body)
except Exception as e:
    print(e)